In [1]:
!pip install -U transformers
!pip install -U datasets
!pip install -U accelerate
!pip install -U peft
!pip install -U trl
!pip install -U bitsandbytes
!pip install huggingface_hub[hf_xet]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 23.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12

In [2]:
from huggingface_hub import login
from google.colab import userdata

hf_token = userdata.get("HF_TOKEN")

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load tokenizer & model
# model_id = "meta-llama/Llama-2-7b-hf"
peft_model = "FinGPT/fingpt-mt_llama2-7b_lora"

model_dir = "facebook/opt-350m"

tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [4]:
!nvidia-smi

Thu Jun 19 08:16:36 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P0             26W /   70W |     772MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
train_prompt_style="""
<|im_start|>system<|im_sep|>
Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.
<|im_end|>
<|im_start|>user<|im_sep|>
{}<|im_end|>
<|im_start|>assistant<|im_sep|>
<think>
{}
</think>
{}
"""

In [6]:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    inputs = examples["Open-ended Verifiable Question"]
    complex_cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs,complex_cots,outputs):
        text = train_prompt_style.format(input,cot,output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

In [7]:
from datasets import load_dataset

dataset = load_dataset(
    "TheFinAI/Fino1_Reasoning_Path_FinQA", split="train[0:1000]", trust_remote_code=True
)
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)
dataset["text"][20]

README.md:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5499 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

"\n<|im_start|>system<|im_sep|>\nBelow is an instruction that describes a task, paired with an input that provides further context.\nWrite a response that appropriately completes the request.\nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n<|im_end|>\n<|im_start|>user<|im_sep|>\nPlease answer the given financial question based on the context.\nContext: american tower corporation and subsidiaries notes to consolidated financial statements 2014 ( continued ) stock-based compensation 2014the company complies with the provisions of sfas no . 148 , 201caccounting for stock-based compensation 2014transition and disclosure 2014an amendment of sfas no . 123 , 201d which provides optional transition guidance for those companies electing to voluntarily adopt the accounting provisions of sfas no . 123 . the company continues to use accounting principles board opinion no . 25 ( apb no . 25 ) , 201caccount

In [8]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [9]:
inference_prompt_style = """
<|im_start|>system<|im_sep|>
Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.
<|im_end|>
<|im_start|>user<|im_sep|>
{}<|im_end|>
<|im_start|>assistant<|im_sep|>
<think>
{}
"""

In [10]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.get_device_name(0))  # Should show the GPU name


True
Tesla T4


In [11]:
question = dataset[20]['Open-ended Verifiable Question']
inputs = tokenizer(
    [inference_prompt_style.format(question, "") + tokenizer.eos_token],
    return_tensors="pt"
).to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=250,
    eos_token_id=tokenizer.eos_token_id,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("<|im_start|>assistant<|im_sep|>")[1])


<think>

</s>I'm not sure if this is a good thing or a bad thing.  I'm not sure if I want to be a part of this.  I'm not sure if I want to be a part of this.  I'm not sure if I want to be a part of this.  I'm not sure if I want to be a part of this.  I'm not sure if I want to be a part of this.  I'm not sure if I want to be a part of this.  I'm not sure if I want to be a part of this.  I'm not sure if I want to be a part of this.  I'm not sure if I want to be a part of this.
I'm not sure if this is a good thing or a bad thing.  I'm not sure if I want to be a part of this.  I'm not sure if I want to be a part of this.  I'm not sure if I want to be a part of this.  I'm not sure if I want to be a part of this.  I'm not sure if I want to be a part of this.  I'm not sure if I want to


In [12]:
from peft import LoraConfig, get_peft_model

# LoRA config
peft_config = LoraConfig(
    lora_alpha=16,                           # Scaling factor for LoRA
    lora_dropout=0.05,                       # Add slight dropout for regularization
    r=64,                                    # Rank of the LoRA update matrices
    bias="none",                             # No bias reparameterization
    task_type="CAUSAL_LM",                   # Task type: Causal Language Modeling
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],  # Target modules for LoRA
)

model = get_peft_model(model, peft_config)

In [13]:
from trl import SFTTrainer
from transformers import TrainingArguments


# Training Arguments
training_arguments = TrainingArguments(
    output_dir="output",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    logging_steps=0.2,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="none"
)

# Initialize the Trainer
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=dataset,
    peft_config=peft_config,
    data_collator=data_collator,
)

Converting train dataset to ChatML:   0%|          | 0/1000 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [15]:
import gc, torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")
# facebook/opt-350m

gc.collect()
torch.cuda.empty_cache()
model.config.use_cache = False

# Set the padding token
tokenizer.pad_token = tokenizer.eos_token

trainer.train()

Step,Training Loss
100,2.567800
200,2.325900
300,2.209400
400,2.229300
500,2.234800


TrainOutput(global_step=500, training_loss=2.313440246582031, metrics={'train_runtime': 805.8123, 'train_samples_per_second': 1.241, 'train_steps_per_second': 0.62, 'total_flos': 1900007115472896.0, 'train_loss': 2.313440246582031})

In [16]:
# prompt: can you save it to this dir ./fine_tuned_mode

# Save the model
output_dir = "./fine_tuned_model"
trainer.save_model(output_dir)

testing fine tuned model

In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Load the base model and tokenizer
model_dir = "facebook/opt-350m"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
base_model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

# Load the fine-tuned LoRA adapter
peft_model_dir = "./fine_tuned_model" # Or wherever you saved your model
model = PeftModel.from_pretrained(base_model, peft_model_dir)

# Merge the LoRA weights with the base model for easier inference
model = model.merge_and_unload()

# Set the padding token if it was set during training
tokenizer.pad_token = tokenizer.eos_token

# Move model to GPU if available
if torch.cuda.is_available():
    model = model.cuda()

In [21]:
# Define your test prompts
test_prompts = [
    "Please answer the given financial question based on the context. Context: In the fiscal year ending December 31, 2023, Company X reported a net income of $500 million. The total revenue for the same period was $2.5 billion. Question: What was the net profit margin for Company X in 2023?",
    # Add more prompts as needed
]

# Prepare prompts with the inference style and tokenize
inference_prompt_style = """
<|im_start|>system<|im_sep|>
Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.
<|im_end|>
<|im_start|>user<|im_sep|>
{}<|im_end|>
<|im_start|>assistant<|im_sep|>
<think>
{}
"""

inputs = [inference_prompt_style.format(prompt, "") + tokenizer.eos_token for prompt in test_prompts]
inputs = tokenizer(inputs, return_tensors="pt", padding=True, truncation=True).to("cuda")


model.eval()
with torch.no_grad(): # Disable gradient calculation for inference
    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=250,  # Adjust as needed
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id, # Use the correct padding token
        use_cache=True,
    )

responses = tokenizer.batch_decode(outputs, skip_special_tokens=True)

for i, response in enumerate(responses):
    print(f"Prompt {i+1}:")
    # Split the response to show only the assistant's part
    assistant_response = response.split("<|im_start|>assistant<|im_sep|>")
    if len(assistant_response) > 1:
        print(assistant_response[1])
    else:
        print(response) # Print the whole response if the split didn't work as expected
    print("-" * 50)

Prompt 1:

<think>

I'm not sure if this is a good idea, but I think it would be a good idea to have a list of all the people who have been killed by the government.                                                                                                                                                                                                                      
--------------------------------------------------


In [20]:
login(token=hf_token)

In [ ]:
# Replace "your-username/your-model-name" with your desired repository ID
repo_id = "your-username/my-fine-tuned-financial-model"

model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)

print(f"Model and tokenizer pushed to https://huggingface.co/{repo_id}")

Remember to replace `"your-username/my-fine-tuned-financial-model"` with your actual Hugging Face username and desired model name.